In [1]:
from sklearn.preprocessing import LabelEncoder
import torch
import pandas as pd
import numpy as np
import pickle
import math

In [2]:
BSC = pd.read_csv('./original_data/Base_station_basic_info.csv')
CLD = pd.read_csv('./original_data/Cell-level_data.csv')
ECD = pd.read_csv('./original_data/Energy_consumption_data.csv')

In [3]:
merged_df = CLD.merge(BSC, on=['BS', 'CellName'], how='inner')
fullmerged_df = merged_df.merge(ECD, on=['BS', 'Time'], how='left') # merge + wyjscie

In [4]:
unique_cells_per_base = fullmerged_df.groupby('BS')['CellName'].nunique()
one_cell_bases = unique_cells_per_base[unique_cells_per_base == 1].index
two_cell_bases = unique_cells_per_base[unique_cells_per_base == 2].index
four_cell_bases = unique_cells_per_base[unique_cells_per_base == 4].index

In [5]:
fullmerged_df_1 = fullmerged_df[fullmerged_df['BS'].isin(one_cell_bases)]
fullmerged_df_2 = fullmerged_df[fullmerged_df['BS'].isin(two_cell_bases)]
fullmerged_df_4 = fullmerged_df[fullmerged_df['BS'].isin(four_cell_bases)]

# print(fullmerged_df_2.value_counts('BS'))

In [6]:
fullmerged_df_1['Time'] = pd.to_datetime(fullmerged_df_1['Time'], format='%m/%d/%Y %H:%M')

fullmerged_df_1['Year'] = fullmerged_df_1['Time'].dt.year
fullmerged_df_1['Day'] = fullmerged_df_1['Time'].dt.day
fullmerged_df_1['Hour'] = fullmerged_df_1['Time'].dt.hour
fullmerged_df_1['Month'] = fullmerged_df_1['Time'].dt.month
# fullmerged_df_1.to_csv('a.csv', index=False)

C:\Users\wpietron\AppData\Local\Temp\ipykernel_18448\291725083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullmerged_df_1['Time'] = pd.to_datetime(fullmerged_df_1['Time'], format='%m/%d/%Y %H:%M')
C:\Users\wpietron\AppData\Local\Temp\ipykernel_18448\291725083.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullmerged_df_1['Year'] = fullmerged_df_1['Time'].dt.year
C:\Users\wpietron\AppData\Local\Temp\ipykernel_18448\291725083.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [7]:
#ewentualne wypelnienie dziur predykcjami i oczywistymi strzalami

with open('./pickle/all_preds_loss_1_64.pkl', 'rb') as file:
    all_preds_dict = pickle.load(file)

fullmerged_df_1.reset_index(inplace=True, drop=True)

for index, row in fullmerged_df_1.iterrows():
    energy = row.Energy
    
    if np.isnan(energy):
        try:
            energy_pred = float(all_preds_dict[(row.BS, str(row.Time))])
            fullmerged_df_1.loc[index, 'Energy'] = energy_pred
        except:
            print("gowno",row.BS, row.Time)

In [8]:
fullmerged_df_1['Energy_1'] = fullmerged_df_1['Energy'].shift(1, fill_value=0)
bs = None
past_energy_cell1_dict = {}
for index, row in fullmerged_df_1.iterrows():
    time = row.Time
    if bs != row.BS:
        bs = row.BS
        fullmerged_df_1.at[index, 'Energy_1'] = 0.0
    elif np.isnan(row.Energy_1):
        fullmerged_df_1.at[index, 'Energy_1'] = fullmerged_df_1.at[index-1, 'Energy_1']
    past_energy_cell1_dict[(bs, time)] = [fullmerged_df_1.at[index, 'Energy_1']]
with open('./pickle/past_energy_cell1_dict.pkl', 'wb') as file:
    pickle.dump(past_energy_cell1_dict, file)

C:\Users\wpietron\AppData\Local\Temp\ipykernel_18448\1141810755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullmerged_df_1['Energy_1'] = fullmerged_df_1['Energy'].shift(1, fill_value=0)


In [9]:
# fullmerged_df_1 = fullmerged_df_1.drop(["CellName", "BS", "Year", "Month", "Time"], axis=1)
fullmerged_df_1 = fullmerged_df_1.drop(["CellName", "Year", "Month"], axis=1)
print(fullmerged_df_1.columns)
new_column_order = ['Time','BS', 'Bandwidth', 'ESMode1','ESMode2','ESMode3','ESMode4','ESMode5','ESMode6','Frequency', 'TXpower', 'load','Mode','Antennas','Day','Hour','RUType','Energy_1','Energy']
fullmerged_df_1 = fullmerged_df_1[new_column_order]
fullmerged_df_1.sort_values(by=['BS', 'Time'], inplace=True)
print(fullmerged_df_1.columns)

Index(['Time', 'BS', 'load', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4',
       'ESMode5', 'ESMode6', 'RUType', 'Mode', 'Frequency', 'Bandwidth',
       'Antennas', 'TXpower', 'Energy', 'Day', 'Hour', 'Energy_1'],
      dtype='object')
Index(['Time', 'BS', 'Bandwidth', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4',
       'ESMode5', 'ESMode6', 'Frequency', 'TXpower', 'load', 'Mode',
       'Antennas', 'Day', 'Hour', 'RUType', 'Energy_1', 'Energy'],
      dtype='object')


In [10]:
label_encoders = {}
object_cols = ['RUType', 'Mode'] 

for col in object_cols:
    le = LabelEncoder()
    fullmerged_df_1[col] = le.fit_transform(fullmerged_df_1[col])
    label_encoders[col] = le

fullmerged_df_1['RUType'].value_counts()

RUType
6     32096
8     27604
9     16087
4      4557
5      4294
7      3999
0       936
2       768
11      648
3       528
1       504
10      264
Name: count, dtype: int64

In [11]:
Encoded_rutype = torch.nn.functional.one_hot(torch.tensor(fullmerged_df_1["RUType"].values).long(), 12)
Encoded_rutype = pd.DataFrame(Encoded_rutype, columns=[f"RUType{i}" for i in range(12)])
# Encoded_rutype.value_counts()

In [12]:
fullmerged_df_1_numpy = fullmerged_df_1.to_numpy()
Encoded_rutype_numpy = Encoded_rutype.to_numpy()
chuj = fullmerged_df_1.columns
fullmerged_df_1 = np.concatenate((fullmerged_df_1_numpy,Encoded_rutype_numpy), axis=1)
fullmerged_df_1 = pd.DataFrame(fullmerged_df_1, columns = chuj.append(Encoded_rutype.columns))

fullmerged_df_1 = fullmerged_df_1.drop(["RUType"], axis=1)

cols = [col for col in fullmerged_df_1 if col != 'Energy'] + ['Energy']
fullmerged_df_1 = fullmerged_df_1[cols]

In [13]:
fullmerged_df_1.to_csv("./prepared_data/One_Cell_merged_with_preds.csv", index=False)
print(fullmerged_df_1.columns)

Index(['Time', 'BS', 'Bandwidth', 'ESMode1', 'ESMode2', 'ESMode3', 'ESMode4',
       'ESMode5', 'ESMode6', 'Frequency', 'TXpower', 'load', 'Mode',
       'Antennas', 'Day', 'Hour', 'Energy_1', 'RUType0', 'RUType1', 'RUType2',
       'RUType3', 'RUType4', 'RUType5', 'RUType6', 'RUType7', 'RUType8',
       'RUType9', 'RUType10', 'RUType11', 'Energy'],
      dtype='object')
